<a href="https://colab.research.google.com/github/anil-chhetri/Miscellaneous/blob/main/Azure%20Databricks/4%20Filter_dataframe_using_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = (SparkSession.builder
        .master("local")
        .appName("Colab")
        .config('spark.ui.port', '4050')
        .getOrCreate())

In [2]:
from pyspark.sql import Row
from pyspark.sql import functions as f

import datetime 
import pandas as pd

In [3]:
users = [
    {
        "id": 1,
        "first_name": "Corrie",
        "last_name": "Van den Oord",
        "email": "cvandenoord0@etsy.com",
        "gender": "male",
        "current_city": "Dallas",
        "phone_numbers": Row(mobile="+1 234 567 8901", home="+1 234 567 8911"),
        "courses": [1, 2],
        "is_customer": True,
        "amount_paid": 1000.55,
        "customer_from": datetime.date(2021, 1, 15),
        "last_updated_ts": datetime.datetime(2021, 2, 10, 1, 15, 0)
    },
    {
        "id": 2,
        "first_name": "Nikolaus",
        "last_name": "Brewitt",
        "email": "nbrewitt1@dailymail.co.uk",
        "gender": "male",
        "current_city": "Houston",
        "phone_numbers":  Row(mobile="+1 234 567 8923", home="1 234 567 8934"),
        "courses": [3],
        "is_customer": True,
        "amount_paid": 900.0,
        "customer_from": datetime.date(2021, 2, 14),
        "last_updated_ts": datetime.datetime(2021, 2, 18, 3, 33, 0)
    },
    {
        "id": 3,
        "first_name": "Orelie",
        "last_name": "Penney",
        "email": "openney2@vistaprint.com",
        "gender": "female",
        "current_city": "",
        "phone_numbers": Row(mobile="+1 714 512 9752", home="+1 714 512 6601"),
        "courses": [2, 4],
        "is_customer": True,
        "amount_paid": 850.55,
        "customer_from": datetime.date(2021, 1, 21),
        "last_updated_ts": datetime.datetime(2021, 3, 15, 15, 16, 55)
    },
    {
        "id": 4,
        "first_name": "Ashby",
        "last_name": "Maddocks",
        "email": "amaddocks3@home.pl",
        "gender": "male",
        "current_city": "San Fransisco",
        "phone_numbers": Row(mobile=None, home=None),
        "courses": [],
        "is_customer": False,
        "amount_paid": None,
        "customer_from": None,
        "last_updated_ts": datetime.datetime(2021, 4, 10, 17, 45, 30)
    },
    {
        "id": 5,
        "first_name": "Kurt",
        "last_name": "Rome",
        "email": "krome4@shutterfly.com",
        "gender": "female",
        "current_city": None,
        "phone_numbers": Row(mobile="+1 817 934 7142", home=None),
        "courses": [],
        "is_customer": False,
        "amount_paid": None,
        "customer_from": None,
        "last_updated_ts": datetime.datetime(2021, 4, 2, 0, 55, 18)
    }
]

In [4]:
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', False)

In [5]:
df = spark.createDataFrame(pd.DataFrame(users))
df.createOrReplaceTempView('employees')

In [6]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- current_city: string (nullable = true)
 |-- phone_numbers: struct (nullable = true)
 |    |-- mobile: string (nullable = true)
 |    |-- home: string (nullable = true)
 |-- courses: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- is_customer: boolean (nullable = true)
 |-- amount_paid: double (nullable = true)
 |-- customer_from: date (nullable = true)
 |-- last_updated_ts: timestamp (nullable = true)



In [7]:
df.show(truncate=False)

+---+----------+------------+-------------------------+------+-------------+----------------------------------+-------+-----------+-----------+-------------+-------------------+
|id |first_name|last_name   |email                    |gender|current_city |phone_numbers                     |courses|is_customer|amount_paid|customer_from|last_updated_ts    |
+---+----------+------------+-------------------------+------+-------------+----------------------------------+-------+-----------+-----------+-------------+-------------------+
|1  |Corrie    |Van den Oord|cvandenoord0@etsy.com    |male  |Dallas       |{+1 234 567 8901, +1 234 567 8911}|[1, 2] |true       |1000.55    |2021-01-15   |2021-02-10 01:15:00|
|2  |Nikolaus  |Brewitt     |nbrewitt1@dailymail.co.uk|male  |Houston      |{+1 234 567 8923, 1 234 567 8934} |[3]    |true       |900.0      |2021-02-14   |2021-02-18 03:33:00|
|3  |Orelie    |Penney      |openney2@vistaprint.com  |female|             |{+1 714 512 9752, +1 714 512 6601}

In [8]:
help(df.filter)

Help on method filter in module pyspark.sql.dataframe:

filter(condition) method of pyspark.sql.dataframe.DataFrame instance
    Filters rows using the given condition.
    
    :func:`where` is an alias for :func:`filter`.
    
    .. versionadded:: 1.3.0
    
    Parameters
    ----------
    condition : :class:`Column` or str
        a :class:`Column` of :class:`types.BooleanType`
        or a string of SQL expression.
    
    Examples
    --------
    >>> df.filter(df.age > 3).collect()
    [Row(age=5, name='Bob')]
    >>> df.where(df.age == 2).collect()
    [Row(age=2, name='Alice')]
    
    >>> df.filter("age > 3").collect()
    [Row(age=5, name='Bob')]
    >>> df.where("age = 2").collect()
    [Row(age=2, name='Alice')]



In [9]:
# get the record where id = 1
df.filter('id = 1').show()
df.filter(f.col('id') == 1).show()
spark.sql('select * from employees where id = 1').show()

##using where is same as filter
df.where('id = 1').show()

+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|gender|current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|  male|      Dallas|{+1 234 567 8901,...| [1, 2]|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|
+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+

+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|g

## using conditiona and operator with filter.

In [10]:
# get all customer records
df.filter('is_customer = true').show()
df.filter(f.col('is_customer') == True).show()
spark.sql("select * from employees where is_customer = 'true'").show()

+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|gender|current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|  male|      Dallas|{+1 234 567 8901,...| [1, 2]|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|
|  2|  Nikolaus|     Brewitt|nbrewitt1@dailyma...|  male|     Houston|{+1 234 567 8923,...|    [3]|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|
|  3|    Orelie|      Penney|openney2@vistapri...|female|            |{+1 714 512 9752,...| [2, 4]|       true|     850.55|   2021-01-21|2021-03-15 15:16:55|
+---+----------+------------+--------------------+--

In [11]:
# get user from Dallas

df.filter("current_city = 'Dallas'").show()
df.filter(f.lower(f.col('current_city')) == 'dallas').show()
spark.sql("select * from employees where lower(current_city) = 'dallas'").show()

+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|gender|current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|  male|      Dallas|{+1 234 567 8901,...| [1, 2]|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|
+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+

+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|g

In [12]:
# get customer who paid 900
df.where("amount_paid = 900").show()
df.where(f.col('amount_paid') == 900.0).show()
spark.sql("""
    select * 
    from employees
    where amount_paid = 900.0
 """).show()

+---+----------+---------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|last_name|               email|gender|current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+---------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  2|  Nikolaus|  Brewitt|nbrewitt1@dailyma...|  male|     Houston|{+1 234 567 8923,...|    [3]|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|
+---+----------+---------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+

+---+----------+---------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|last_name|               email|gender|current_city|  

In [13]:
# get customer where amount_paid is not a number
df.filter("isnan(amount_paid) = true").show()
df.where(f.isnan(f.col('amount_paid'))).show()
spark.sql("""
  select * from employees 
  where isnan(amount_paid) = true
 """).show()

+---+----------+---------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|last_name|               email|gender| current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+---------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  4|     Ashby| Maddocks|  amaddocks3@home.pl|  male|San Fransisco|        {null, null}|     []|      false|        NaN|         null|2021-04-10 17:45:30|
|  5|      Kurt|     Rome|krome4@shutterfly...|female|         null|{+1 817 934 7142,...|     []|      false|        NaN|         null|2021-04-02 00:55:18|
+---+----------+---------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+

+---+----------+---------+--------------------+------+---------

In [14]:
df.show()

+---+----------+------------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|gender| current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+------------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|  male|       Dallas|{+1 234 567 8901,...| [1, 2]|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|
|  2|  Nikolaus|     Brewitt|nbrewitt1@dailyma...|  male|      Houston|{+1 234 567 8923,...|    [3]|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|
|  3|    Orelie|      Penney|openney2@vistapri...|female|             |{+1 714 512 9752,...| [2, 4]|       true|     850.55|   2021-01-21|2021-03-15 15:16:55|
|  4|     Ashby|    Maddocks|  amaddocks3@home

In [15]:
# get all the user who are not living in dallas
(df.where((f.col('current_city') != 'Dallas') | (df['current_city'].isNull()))).show()
(
    df.where("current_city != 'Dallas' or current_city is not null")
).show()
spark.sql("""
select * from employees where coalesce(current_city, '') <> 'Dallas'
 """).show()

+---+----------+---------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|last_name|               email|gender| current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+---------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  2|  Nikolaus|  Brewitt|nbrewitt1@dailyma...|  male|      Houston|{+1 234 567 8923,...|    [3]|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|
|  3|    Orelie|   Penney|openney2@vistapri...|female|             |{+1 714 512 9752,...| [2, 4]|       true|     850.55|   2021-01-21|2021-03-15 15:16:55|
|  4|     Ashby| Maddocks|  amaddocks3@home.pl|  male|San Fransisco|        {null, null}|     []|      false|        NaN|         null|2021-04-10 17:45:30|
|  5|      Kurt|     Rome|krome4@shutterfly...|female|         n

In [16]:
# get all users where city name is not empty.
df.filter("current_city <> '' and is_customer = true").show()
df.filter((f.col('current_city') != '') & (f.col('is_customer') == 'true')).show()
spark.sql(""" 
  select *
  from employees 
  where is_customer = true and current_city <> ''
""").show()

+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|gender|current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|  male|      Dallas|{+1 234 567 8901,...| [1, 2]|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|
|  2|  Nikolaus|     Brewitt|nbrewitt1@dailyma...|  male|     Houston|{+1 234 567 8923,...|    [3]|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|
+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+

+---+----------+------------+--------------------+-

In [32]:
# get user and email whose lastupdate ts is between 2021-02-15 and 2021-03-15
(df.filter(f.col('last_updated_ts').between('2021-02-15', '2021-03-15 23:59:59')).select('id', 'email', 'last_updated_ts')).show()
(df.where("last_updated_ts between '2021-02-15' and '2021-03-15 23:59:59'").select('id', 'email', 'last_updated_ts')).show()
spark.sql("""
select id, email, last_updated_ts from employees
where last_updated_ts between '2021-02-15' and '2021-03-15 23:59:59'
 """).show()

+---+--------------------+-------------------+
| id|               email|    last_updated_ts|
+---+--------------------+-------------------+
|  2|nbrewitt1@dailyma...|2021-02-18 03:33:00|
|  3|openney2@vistapri...|2021-03-15 15:16:55|
+---+--------------------+-------------------+

+---+--------------------+-------------------+
| id|               email|    last_updated_ts|
+---+--------------------+-------------------+
|  2|nbrewitt1@dailyma...|2021-02-18 03:33:00|
|  3|openney2@vistapri...|2021-03-15 15:16:55|
+---+--------------------+-------------------+

+---+--------------------+-------------------+
| id|               email|    last_updated_ts|
+---+--------------------+-------------------+
|  2|nbrewitt1@dailyma...|2021-02-18 03:33:00|
|  3|openney2@vistapri...|2021-03-15 15:16:55|
+---+--------------------+-------------------+



In [37]:
# get user whose payment is in range 850 and 900
(df.filter('amount_paid between 850 and 900').show())
(df.filter(f.col('amount_paid').between(850, 900))).show()
spark.sql(""" 
select * from employees where amount_paid between 850 and 900
""").show()

+---+----------+---------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|last_name|               email|gender|current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+---------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  2|  Nikolaus|  Brewitt|nbrewitt1@dailyma...|  male|     Houston|{+1 234 567 8923,...|    [3]|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|
|  3|    Orelie|   Penney|openney2@vistapri...|female|            |{+1 714 512 9752,...| [2, 4]|       true|     850.55|   2021-01-21|2021-03-15 15:16:55|
+---+----------+---------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+

+---+----------+---------+--------------------+------+------------+--

In [40]:
# get all the users whose city is not null
df.filter('current_city is not null').show()

+---+----------+------------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|gender| current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+------------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|  male|       Dallas|{+1 234 567 8901,...| [1, 2]|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|
|  2|  Nikolaus|     Brewitt|nbrewitt1@dailyma...|  male|      Houston|{+1 234 567 8923,...|    [3]|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|
|  3|    Orelie|      Penney|openney2@vistapri...|female|             |{+1 714 512 9752,...| [2, 4]|       true|     850.55|   2021-01-21|2021-03-15 15:16:55|
|  4|     Ashby|    Maddocks|  amaddocks3@home

In [42]:
df.filter(f.col('current_city').isNull()).show()

+---+----------+---------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|last_name|               email|gender|current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+---------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  5|      Kurt|     Rome|krome4@shutterfly...|female|        null|{+1 817 934 7142,...|     []|      false|        NaN|         null|2021-04-02 00:55:18|
+---+----------+---------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+



In [44]:
df.filter(f.col('customer_from').isNull()).show()

+---+----------+---------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|last_name|               email|gender| current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+---------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  4|     Ashby| Maddocks|  amaddocks3@home.pl|  male|San Fransisco|        {null, null}|     []|      false|        NaN|         null|2021-04-10 17:45:30|
|  5|      Kurt|     Rome|krome4@shutterfly...|female|         null|{+1 817 934 7142,...|     []|      false|        NaN|         null|2021-04-02 00:55:18|
+---+----------+---------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+



In [47]:
# get users whose city is either houston and dallas
df.where((f.col('current_city') == 'Houston') | (f.col('current_city') == 'Dallas')).show()

+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|gender|current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|  male|      Dallas|{+1 234 567 8901,...| [1, 2]|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|
|  2|  Nikolaus|     Brewitt|nbrewitt1@dailyma...|  male|     Houston|{+1 234 567 8923,...|    [3]|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|
+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+



In [54]:
df.where(f.col('amount_paid') > 900).show()
df.where('amount_paid > 900').show()

+---+----------+------------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|gender| current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+------------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|  male|       Dallas|{+1 234 567 8901,...| [1, 2]|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|
|  4|     Ashby|    Maddocks|  amaddocks3@home.pl|  male|San Fransisco|        {null, null}|     []|      false|        NaN|         null|2021-04-10 17:45:30|
|  5|      Kurt|        Rome|krome4@shutterfly...|female|         null|{+1 817 934 7142,...|     []|      false|        NaN|         null|2021-04-02 00:55:18|
+---+----------+------------+-----------------

In [57]:
# get male customer
df.where("is_customer = true and gender = 'male'").show()

+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|gender|current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|  male|      Dallas|{+1 234 567 8901,...| [1, 2]|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|
|  2|  Nikolaus|     Brewitt|nbrewitt1@dailyma...|  male|     Houston|{+1 234 567 8923,...|    [3]|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|
+---+----------+------------+--------------------+------+------------+--------------------+-------+-----------+-----------+-------------+-------------------+



In [58]:
#get user who are not customer or city contain empty string
df.filter((f.col('is_customer') == 'false')| (f.col('current_city') != '')).show()

+---+----------+------------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|gender| current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+------------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|  male|       Dallas|{+1 234 567 8901,...| [1, 2]|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|
|  2|  Nikolaus|     Brewitt|nbrewitt1@dailyma...|  male|      Houston|{+1 234 567 8923,...|    [3]|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|
|  4|     Ashby|    Maddocks|  amaddocks3@home.pl|  male|San Fransisco|        {null, null}|     []|      false|        NaN|         null|2021-04-10 17:45:30|
|  5|      Kurt|        Rome|krome4@shutterfly

In [23]:
df.show()

+---+----------+------------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|gender| current_city|       phone_numbers|courses|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+------------+--------------------+------+-------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  1|    Corrie|Van den Oord|cvandenoord0@etsy...|  male|       Dallas|{+1 234 567 8901,...| [1, 2]|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|
|  2|  Nikolaus|     Brewitt|nbrewitt1@dailyma...|  male|      Houston|{+1 234 567 8923,...|    [3]|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|
|  3|    Orelie|      Penney|openney2@vistapri...|female|             |{+1 714 512 9752,...| [2, 4]|       true|     850.55|   2021-01-21|2021-03-15 15:16:55|
|  4|     Ashby|    Maddocks|  amaddocks3@home